In [339]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [340]:
# Import data set
df = pd.read_csv('../data/artDataset.csv')
df = df.drop(columns = 'Unnamed: 0')

In [341]:
df

,price,artist,title,yearCreation,signed,condition,period,movement
0,28.500 USD,Tommaso Ottieri,Bayreuth Opera,2021,Signed on verso,This work is in excellent condition.,Contemporary,Baroque
1,3.000 USD,Pavel Tchelitchew,Drawings of the Opera,First Half 20th Century,Signed and titled,Not examined out of frame.No obvious signs of ...,Post-War,Surrealism
2,5.000 USD,Leo Gabin,Two on Sidewalk,2016,"Signed, titled and dated on verso",This work is in excellent condition.,Contemporary,Abstract
3,5.000 USD,Matthias Dornfeld,Blumenszene,2010,"Signed, titled and dated on the reverse with t...",This work is in excellent condition.There is m...,Contemporary,Abstract
4,2.500 USD,Alexis Marguerite Teplin,Feverish Embarkation,2001,Signed on verso,This work is in excellent condition.,Contemporary,Abstract
...,...,...,...,...,...,...,...,...
749,680 USD,Jane Kent,Miracle Grow #17,2012,Signed and dated on lower right.,Not examined out of frame.No obvious signs of ...,Contemporary,Abstract
750,1.275 USD,Gary Bower,Rolph Series,1970,[nan],Not examined out of frame.Significant undulati...,Contemporary,Geometric Abstraction
751,680 USD,Jane Kent,Untitled,2012,[nan],Not examined out of frame.No apparent imperfec...,Contemporary,Geometric Abstraction
752,1.275 USD,T. L. Solien,Juniper,1986,[nan],Not examined outside of frame.Pinholes at edge...,Contemporary,Abstract


In [342]:
df['price']

0      28.500 USD
1       3.000 USD
2       5.000 USD
3       5.000 USD
4       2.500 USD
          ...    
749       680 USD
750     1.275 USD
751       680 USD
752     1.275 USD
753       680 USD
Name: price, Length: 754, dtype: object

## **price**

They are string values, so we have to convert them to numeric values.

In [343]:
# Remove 'USD' and convert to float
# Remove thousand separators and ' USD', then convert to float
df['price'] = df['price'].str.replace('.', '', regex=False).str.replace(' USD', '', regex=False).astype(float)

## **artist**

Usually, artists are a big indicator of an art piece's price. However, we have a very reduced data set, in which for example we have a lot of pieces from unique artists, but also artists with multiple pieces. If we one-hot encode every artist, we will end up with high-dimensionality issues, which we want to avoid (some techniques do not perform well with high-dimensionality).

We have several options to solve this problem:
- Frecuency encoding: it's simple but does not capture the relationship between the artist and the price.
- Grouping artists: keep the top N artists and grouping the rest into an "other" category.
- Mean target encoding: it's a more complex technique that captures the relationship between the artist and the price, but it's also dangerous to data leakage.

We will use the grouping artists technique. Since there are a lot of 'Rare' artists, we will group them into an 'Rare' category. Artists with more than 1 piece will be grouped into an 'Frequent' category. This a baseline approach, and since we are aware that artist is an important predictor, we can try to improve it later.

In [344]:
# Let's see how many unique values are
print(f'There are {len(df['artist'].unique())} unique artists in this data set.')

# There is one missing value, so let's delete it
df.dropna(subset=['artist'], inplace=True)

# Get a peek
artist_counts = df['artist'].value_counts()

# Group artists based on frecuency of appearance
df['artist_grouped'] = df['artist'].apply(lambda x: 'Frequent' if artist_counts[x] > 1 else 'Rare')


There are 455 unique artists in this data set.


## **title**

Title will not be considered initially, as it does not bring direct value to the predictions. However, some insights could be obtained from the title, like the length, the presence of certain words, etc.

## **yearCreation**

In [345]:
df['yearCreation'].unique()

array(['2021', 'First Half 20th Century ', '2016', '2010', '2001', '1977',
       '2007', '1993', '2014', '1973', '[nan]', '2008', '2015', '2012',
       '2002', '1992', '2020', '1992 - 2004', '1991', '2018', '1920',
       'Mid 20th Century ', 'Second Half 20th Century ', '2003', '2000',
       '2005', '1999', '1958', '19th Century ', 'Circa 1970 - 1979',
       '1877', 'Circa 1877', 'Second Half 19th Century ', '2004', '2013',
       '2006', '1990', 'Circa 1925', '1935', '1928', '2022',
       '1961, printed in 2010', '1994',
       '3D printed using ABS, PLA plastics, resin, automobile paints, etched brass, dry transfers, acrylic mirror, batik fabric, quartz clock',
       '1992 - 1993', '1996', '2017', '2019', '2011', '1948', '1971',
       '1960', '1972', '1984', 'Late 20th Century ', '1981', '1976',
       '1950', '1896', '2002 - 2020', '1965 - 2018', '1975', '1968',
       '1970', '1969', '1998', '1996-2003', '1940', '1988',
       'Late 19th Century ', '1892', '1947', '1995', '

In [346]:
# Function to deal with every specific year case. For those ranges and Circa, approximate the year.
def clean_year(year_str):
    if pd.isna(year_str) or year_str == '[nan]':
        return None
    
    # Remove the material description entry
    if '3D printed' in str(year_str):
        return None
    
    # Handle simple years
    if str(year_str).isdigit():
        return int(year_str)
    
    # Handle 'Circa' dates
    if 'Circa' in str(year_str):
        year = ''.join(filter(str.isdigit, year_str))
        return int(year) if year else None
    
    # Handle date ranges (take the earliest year)
    if '-' in str(year_str) or ' - ' in str(year_str):
        years = re.findall(r'\d{4}', year_str)
        return int(years[0]) if years else None
    
    # Handle century descriptions with half periods
    if 'Century' in str(year_str):
        if '19th' in year_str:
            if 'First Half' in year_str:
                return 1825  # middle of first half of 19th century
            elif 'Second Half' in year_str:
                return 1875  # middle of second half of 19th century
            else:
                return 1850  # middle of 19th century
        elif '20th' in year_str:
            if 'First Half' in year_str:
                return 1925  # middle of first half of 20th century
            elif 'Second Half' in year_str:
                return 1975  # middle of second half of 20th century
            elif 'Early' in year_str:
                return 1915  # early 20th century
            elif 'Late' in year_str:
                return 1985  # late 20th century
            elif 'Mid' in year_str:
                return 1950  # mid 20th century
            else:
                return 1950  # middle of 20th century
        elif '21st' in year_str:
            return 2000  # beginning of 21st century
    
    # Handle comma-separated dates (take the first year)
    if ',' in str(year_str):
        years = re.findall(r'\d{4}', year_str)
        return int(years[0]) if years else None
    
    # Try to extract any 4-digit year
    years = re.findall(r'\d{4}', str(year_str))
    return int(years[0]) if years else None

# Apply the cleaning function
df['year_cleaned'] = df['yearCreation'].apply(clean_year)

# Check the results
print("Sample of cleaned years:")
print(pd.DataFrame({'original': df['yearCreation'], 'cleaned': df['year_cleaned']}).head(20))

Sample of cleaned years:
                    original  cleaned
0                       2021   2021.0
1   First Half 20th Century    1925.0
2                       2016   2016.0
3                       2010   2010.0
4                       2001   2001.0
5                       1977   1977.0
6                       2007   2007.0
7                       1993   1993.0
8                       2014   2014.0
9                       1993   1993.0
10                      2014   2014.0
11                      1973   1973.0
12                     [nan]      NaN
13                      2008   2008.0
14                      2015   2015.0
15                      2015   2015.0
16                      2012   2012.0
17                      2002   2002.0
18                      2008   2008.0
19                      2002   2002.0


In [347]:
# Delete the NaN observations, since year is a relevant feature
df.dropna(subset=['year_cleaned'], inplace=True)

## **period and movement**

In [348]:
# First we focus on the period
df['period'].value_counts()

# Delete since there is a nan value
df = df[df['period']!='[nan]']

# Now let's do the same with the movement
df['movement'].unique()
df['movement'].value_counts()

# Delete since there is a nan value
df = df[df['movement']!='[nan]']

## **signed**

In [349]:
print(f'There are {len(df['signed'].unique())} unique values in the signed category')
df['signed'].value_counts()

# Clean text
df['signed_clean'] = df['signed'].astype(str).str.lower().str.strip()

# Binary feature extraction
# Basically create multiple categories to see if each observation has a certain characteristic
df['is_signed'] = df['signed_clean'].str.contains('signed').astype(int)
df['is_dated'] = df['signed_clean'].str.contains('dated').astype(int)
df['is_numbered'] = df['signed_clean'].str.contains('numbered').astype(int)
df['is_titled'] = df['signed_clean'].str.contains('titled').astype(int)
df['is_recto'] = df['signed_clean'].str.contains('recto').astype(int)
df['is_verso'] = df['signed_clean'].str.contains('verso').astype(int)
df['signed_pencil'] = df['signed_clean'].str.contains('pencil').astype(int)
df['signed_paint'] = df['signed_clean'].str.contains('paint').astype(int)
df['signed_ink'] = df['signed_clean'].str.contains('ink').astype(int)
df['is_embossed_or_stamped'] = df['signed_clean'].str.contains('embossed|stamped').astype(int)
df['is_certificate'] = df['signed_clean'].str.contains('certificate|authenticity').astype(int)

There are 380 unique values in the signed category


## **condition**

In [350]:
print(f'There are {len(df['condition'].unique())} unique values in the condition category')

# Clean the text
df['condition_clean'] = df['condition'].astype(str).str.lower().str.strip()

df['is_examined'] = ~df['condition_clean'].str.contains('not examined')
df['is_excellent'] = df['condition_clean'].str.contains('excellent condition').astype(int)
df['is_very_good'] = df['condition_clean'].str.contains('very good condition').astype(int)
df['is_good'] = df['condition_clean'].str.contains('good condition').astype(int)
df['is_fair_or_poor'] = df['condition_clean'].str.contains('fair|moderate wear|poor').astype(int)
df['has_damages'] = df['condition_clean'].str.contains('hole|tear|crack|loss|scratch|abrasion|dent|rippling|crease|discoloration|stain|mark').astype(int)
df['has_restoration_or_retouching'] = df['condition_clean'].str.contains('retouch|restor|inpaint').astype(int)
df['has_uv_analysis'] = df['condition_clean'].str.contains('under uv').astype(int)
df['unexamined'] = df['condition_clean'].str.contains('not examined').astype(int)
df['has_frame_wear'] = df['condition_clean'].str.contains('frame').astype(int)

There are 359 unique values in the condition category


## **final dataset**

In [351]:
df_new = df.copy()
df_new = df_new.drop(columns = ['artist','title','yearCreation','signed','condition','signed_clean'])
df_new.columns

Index(['price', 'period', 'movement', 'artist_grouped', 'year_cleaned',
       'is_signed', 'is_dated', 'is_numbered', 'is_titled', 'is_recto',
       'is_verso', 'signed_pencil', 'signed_paint', 'signed_ink',
       'is_embossed_or_stamped', 'is_certificate', 'condition_clean',
       'is_examined', 'is_excellent', 'is_very_good', 'is_good',
       'is_fair_or_poor', 'has_damages', 'has_restoration_or_retouching',
       'has_uv_analysis', 'unexamined', 'has_frame_wear'],
      dtype='object')

Now we one-hot encode the categorical features left: period, movement, artist_grouped

In [352]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode categorical features: 'period', 'movement', and 'artist_grouped'
# Initialize the encoder
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

# Fit and transform the categorical features
categorical_features = ['period', 'movement', 'artist_grouped']
encoded_features = encoder.fit_transform(df_new[categorical_features])

# Get the feature names after encoding
feature_names = encoder.get_feature_names_out(categorical_features)

# Create a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=feature_names, index=df_new.index)

# Display the first few rows of the encoded features
print("Shape of encoded features:", encoded_features.shape)
print("\nFirst few rows of encoded features:")
encoded_df.head()

Shape of encoded features: (731, 36)

First few rows of encoded features:


,period_Contemporary,period_Modern,period_Post-War,movement_Abstract Expressionism,movement_Art Brut,movement_Art Deco,movement_Art Nouveau,movement_Baroque,movement_Conceptual,movement_Cubism,...,movement_Post-Minimalism,movement_Punk,"movement_Punk, Young British Artists, Abstract",movement_Realism,movement_Romanticism,movement_Social Realism,movement_Street Art,movement_Surrealism,movement_Traditional,artist_grouped_Rare
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [353]:
# Now, we merge the two data sets
df_processed = df_new.copy()

# Then concatenate with the encoded features
df_processed = pd.concat([df_processed, encoded_df], axis=1)

# Drop the original categorical columns if you don't need them anymore
df_processed = df_processed.drop(columns=categorical_features)

In [354]:
# Display the first few rows of the processed DataFrame
df_processed

,price,year_cleaned,is_signed,is_dated,is_numbered,is_titled,is_recto,is_verso,signed_pencil,signed_paint,...,movement_Post-Minimalism,movement_Punk,"movement_Punk, Young British Artists, Abstract",movement_Realism,movement_Romanticism,movement_Social Realism,movement_Street Art,movement_Surrealism,movement_Traditional,artist_grouped_Rare
0,28500.0,2021.0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3000.0,1925.0,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,5000.0,2016.0,1,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5000.0,2010.0,1,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2500.0,2001.0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,680.0,2012.0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
750,1275.0,1970.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
751,680.0,2012.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752,1275.0,1986.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [355]:
# Display the first few rows of the processed DataFrame
df_processed.drop(columns='condition_clean', inplace=True)

In [356]:
df_processed

,price,year_cleaned,is_signed,is_dated,is_numbered,is_titled,is_recto,is_verso,signed_pencil,signed_paint,...,movement_Post-Minimalism,movement_Punk,"movement_Punk, Young British Artists, Abstract",movement_Realism,movement_Romanticism,movement_Social Realism,movement_Street Art,movement_Surrealism,movement_Traditional,artist_grouped_Rare
0,28500.0,2021.0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3000.0,1925.0,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,5000.0,2016.0,1,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5000.0,2010.0,1,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2500.0,2001.0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,680.0,2012.0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
750,1275.0,1970.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
751,680.0,2012.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752,1275.0,1986.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [357]:
# Save data in a new csv file
df_processed.to_csv('../data/artDataset_preprocessed.csv', index=False)